# imports

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from cvxopt import matrix, solvers
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle

# data import

In [11]:
cd /content/drive/My Drive/OPT_HW2

/content/drive/My Drive/OPT_HW2


In [0]:
data_path = '/content/drive/My Drive/OPT_HW2'

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 24 11:23:09 2019

@author: marco
"""

"""
this is the code you need to run to import the data. 
You just have to change line 40 putting the correct path.
"""

import numpy as np

def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist(data_path+'/Data', kind='train')


"""
We are only interested in the items with label 2, 4 and 6.
Only a subset of 1000 samples per class will be used.
"""
indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:1000,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:1000].astype('float64') 


"""
To train a SVM, you have to convert the labels of the two classes of interest into '+1' and '-1'.
"""


"\nTo train a SVM, you have to convert the labels of the two classes of interest into '+1' and '-1'.\n"

In [0]:
SEED = 1848399
random.seed(SEED)
np.random.seed(SEED)

In [0]:
# converting labels of classes 2 and 4 into +1 and -1, respectively
yLabel2 = yLabel2 / 2.0
yLabel4 = yLabel4 / -4.0

In [0]:
xLabel24 = np.concatenate((xLabel2, xLabel4))
yLabel24 = np.concatenate((yLabel2, yLabel4))

In [0]:
# train 70%, test 30%
x_train24, x_test24, y_train24, y_test24 = train_test_split(
    xLabel24, yLabel24, train_size=0.7, random_state=SEED)

In [0]:
# scaling data
scaler = StandardScaler()
scaler.fit(x_train24)
x_train24 = scaler.transform(x_train24)
x_test24 = scaler.transform(x_test24)

# main

**hyperparameters from grid-search:**

C_params = [0.01, 0.1, 1, 10, 100, 1000, 10000]

Gamma_params = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]

**Best ones on validation set with 5-fold CV:**

C - Gamma - Accuracy

100__0.001 0.8357142857142857

1000__0.001 0.8357142857142857

10000__0.001 0.8357142857142857



In [27]:
# best HPs:
gamma = 0.001 #for rbf kernel
c = 100
res_test = SVM_prediction(x_train24, y_train24, gamma, c, x_test24, y_test24)
acc_test = accuracy_score(y_test24, res_test[1])
conf_matrix = confusion_matrix(y_test24, res_test[1]) # first row tn, fp; second row fn, tp
print('TEST INFO')
print(c, gamma, acc_test)
print(conf_matrix)

res_train = SVM_prediction(x_train24, y_train24, gamma, c, x_train24, y_train24)
acc_train = accuracy_score(y_train24, res_train[1])
conf_matrix = confusion_matrix(y_train24, res_train[1])
print('TRAIN INFO')
print(c, gamma, acc_train)
print(conf_matrix)

number of nonzero lambda:  1179
TEST INFO
100 0.001 0.895
[[264  29]
 [ 34 273]]
number of nonzero lambda:  1179
TRAIN INFO
100 0.001 1.0
[[707   0]
 [  0 693]]


# functions

In [0]:
def train_val(k, x, y):
    idx = [n for n in range(len(x)) if n != k]
    x_test = x[k]
    y_test = y[k]
    x_train = np.empty([0, x[1].shape[1]])
    y_train = np.empty([0, ])
    for i in idx:
        x_train = np.concatenate((x_train, x[i]))
        y_train = np.concatenate((y_train, y[i]))
    return x_train, y_train, x_test, y_test

In [0]:
# C, x, y, kernel function
def QP(x, y,rbf_gamma,C):
    ''''' Solving SVM nonlinear dual soft:
    min(lambda) = lambdaT * Q * lambda + PT * lambda
    inqualities: G * lambda <= h ==> lambda <= C and -lambda <= 0
    equalitis: A * lambda = b ==> lambdaT * y = 0
    '''''
    # In QP formulation (dual): n variables, 2n+1 constraints (1 equation, 2n inequations)
    n = x.shape[0] # number of samples in data
    #print('x.shape: ', x.shape, 'y.shape: ', y.shape)
    Q = kernel_rbf_Q(x, y, rbf_gamma) # write function for kernal 
    Q = matrix(Q, tc='d') # transforming for solver (n,n)
    e = np.ones((n, 1)) # (n,1)
    P = matrix(-e, tc='d')# (n,1)
    # equalities 
    A = matrix(y.reshape((1,-1)), tc='d') # (#of equalities,n) 
    b = matrix([0.0]) #(#of equalities*1)=(1,1)
    # inequalities
    # 2 inequalities for each sample -lambda <= 0 and lambda <= C
    h = matrix(np.r_[np.zeros((n,1)), np.zeros((n,1)) + C], tc='d') # (2*n,1)
    G = matrix(np.r_[-np.eye(n), np.eye(n)], tc='d') #(n*2,#of ineq for each sample=2)
    #print('Q: ', Q.size)
    #print('P: ', P.size)
    #print('G: ', G.size)
    #print('h: ', h.size)
    #print('A: ', A.size)
    #print('b: ', b.size)
    solvers.options['show_progress'] = False
    solution = solvers.qp(Q, P, G, h, A, b)
    if solution['status'] != 'optimal':
        print('Not PSD!')
    else:
        return solution

In [0]:
def kernel_rbf_Q(x,y,rbf_gamma):
    P = y.shape[0]
    Q = np.zeros((P,P))
    for i in range(P):
        for j in range(P):
            Q[i,j] = y[i] * y[j] * np.exp(-rbf_gamma * np.linalg.norm(x[i] - x[j]) ** 2) 
    return Q

In [0]:
def kernel_rbf_elem(x,y,rbf_gamma):
    K = rbf_kernel(X=x, Y=y, gamma=rbf_gamma)
    return K

In [0]:
def prediction_full_P(x,y,lambda_vector, b, x_test,rbf_gamma):
    predictions_y = np.zeros((x_test.shape[0],))
    for t in range(x_test.shape[0]):
        decision_func_val = 0
        for i in range(x.shape[0]):
            k = kernel_rbf_elem(x[i].reshape(1, -1), x_test[t].reshape(1, -1), rbf_gamma)
            decision_func_val = decision_func_val + (lambda_vector[i][0] * y[i] * k)
        predictions_y[t] = np.sign(decision_func_val + b)
    return predictions_y

In [0]:
def optimal_b_star(x,y,lambda_vector, rbf_gamma):
    ''''' Whenever lambda optimal is strictly positive,
    b should be a unique value for all the training samples
    '''''
    sum_mult = 0
    i = np.argmax(lambda_vector) # index of nonzero element of lambda with max gap
    for j in range(x.shape[0]):
        k = kernel_rbf_elem(x[j].reshape(1, -1), x[i].reshape(1, -1), rbf_gamma)
        mult = lambda_vector[j][0] * y[j] * k
        sum_mult = sum_mult + mult
    b = (1 - y[i] * sum_mult)[0][0]/y[i]
    return b

In [0]:
def SVM_prediction(x_train, y_train, rbf_gamma, C, x_test, y_test):
    result = []
    QPsolution = QP(x_train, y_train, rbf_gamma, C)
    #print('QPsolution:', QPsolution)
    lambda_vector = np.array(QPsolution['x'])
    #filtering lambda vector is striclty positive (1e-7 is counted as 0)
    lambda_vector[lambda_vector < 1e-7] = 0
    print('number of nonzero lambda: ', np.count_nonzero(lambda_vector))
    b = optimal_b_star(x_train, y_train, lambda_vector, rbf_gamma)
    y_pred = prediction_full_P(x_train, y_train, lambda_vector, b, x_test, rbf_gamma)
    result.extend((QPsolution, y_pred))
    return result

# K-fold (IF YOU RUN IT, IT TAKES YEARS)

In [0]:
k_folds = 5
x_train24, y_train24 = shuffle(x_train24, y_train24, random_state=SEED)
y_train24_sub = np.split(y_train24, k_folds)
x_train24_sub = np.split(x_train24, k_folds)

In [0]:
C_params = [0.01, 0.1, 1, 10, 100, 1000, 10000]
Gamma_params = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]

In [95]:
result_dict = dict()
for c in C_params:
    for gamma in Gamma_params:
        param_key = str(c) + '__' + str(gamma)
        print(param_key)
        # one pair of parameters
        result_k_fold = []
        for k in range(k_folds):
            x_train, y_train, x_test, y_test = train_val(2, x_train24_sub, y_train24_sub)
            res = SVM_prediction(x_train, y_train, gamma, c, x_test, y_test)
            result_k_fold.append(res)
            acc = accuracy_score(y_test, res[1])
            if res[1] < 0.6:
                break
        result_dict.update({param_key:result_k_fold})
print(result_dict)

0.01__1e-05
     pcost       dcost       gap    pres   dres
 0: -4.7204e+02 -4.7989e+01  7e+03  9e+01  1e-14
 1: -4.3227e+01 -2.7426e+01  4e+02  4e+00  1e-14
 2: -8.7584e+00 -2.4880e+01  2e+01  6e-16  2e-15
 3: -1.0079e+01 -1.2600e+01  3e+00  1e-16  9e-16
 4: -1.1146e+01 -1.1204e+01  6e-02  7e-17  9e-16
 5: -1.1174e+01 -1.1175e+01  6e-04  8e-17  7e-16
 6: -1.1175e+01 -1.1175e+01  6e-06  7e-17  6e-16
Optimal solution found.
QPsolution[status]: optimal
number of nonzero lambda:  1120
b:  -0.9788289919077962
acc: 0.525
{'0.01__1e-05': [[-0.9788289919077962, 0.525]]}
0.01__0.0001
     pcost       dcost       gap    pres   dres
 0: -3.3836e+02 -5.5344e+01  8e+03  9e+01  8e-15
 1: -5.5040e+01 -3.2242e+01  6e+02  7e+00  7e-15
 2: -9.9745e+00 -2.6282e+01  3e+01  1e-01  2e-15
 3: -9.7546e+00 -1.4284e+01  5e+00  2e-02  7e-16
 4: -1.0814e+01 -1.1063e+01  3e-01  4e-04  6e-16
 5: -1.0969e+01 -1.0972e+01  3e-03  4e-06  6e-16
 6: -1.0971e+01 -1.0971e+01  3e-05  4e-08  5e-16
 7: -1.0971e+01 -1.0971e+0

# not used functions

In [0]:
def polinomial_kernel(x,p_gamma):
    K = np.zeros((x.shape[0], x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            k = (np.matmul(x[i].T,x[j]) + 1.0) ** p_gamma
            K[i,j] = k
    return K

from sklearn.metrics.pairwise import polynomial_kernel
def auto_kernel_pol(x,y,p_gamma):
    K = polynomial_kernel(X=x, Y=y, degree=p_gamma, gamma=1, coef0=1)
    return K

from numpy import linalg as LA
def rbf_kernel(x,rbf_gamma):
    K = np.zeros((x.shape[0], x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            k = np.exp(-rbf_gamma * (LA.norm(x[i] - x[j])) ** 2)
            K[i,j] = k
    return K

In [0]:
def prediction_pos_P(x,y,lambda_vector, b, x_test,rbf_gamma):
    non_zero_lambda = np.nonzero(lambda_vector)[0]
    predictions_y = np.zeros((x_test.shape[0],))
    for t in range(x_test.shape[0]):
        decision_func_val = 0
        for elem in non_zero_lambda:
            k = auto_kernel_rbf(x[elem].reshape(1, -1), x_test[t].reshape(1, -1), rbf_gamma)
            decision_func_val = decision_func_val + (lambda_vector[elem][0] * y[elem] * k + b)
        predictions_y[t] = np.sign(decision_func_val)
    return predictions_y

In [0]:
def prediction_pos_P1(x,y,lambda_vector, x_test,rbf_gamma):
    non_zero_lambda = np.nonzero(lambda_vector)[0]
    predictions_y = np.zeros((x_test.shape[0],))
    for t in range(x_test.shape[0]):
        decision_func_val = 0
        for elem in non_zero_lambda:
            k = auto_kernel_rbf(x[elem].reshape(1, -1), x_test[t].reshape(1, -1), rbf_gamma)
            b = ((1 - y[elem] * np.matmul(w.T, x[elem])) / y[elem])
            decision_func_val = decision_func_val + (lambda_vector[elem][0] * y[elem] * k + b)
        predictions_y[t] = np.sign(decision_func_val)
    return predictions_y

In [0]:
def optimal_w(x,y,lambda_vector):
    w = np.zeros((x.shape[1], ))
    for i in range(x.shape[0]):
        w = w + lambda_vector[i] * y[i] * x[i]
    return w

In [0]:
def optimal_b(x,y,lambda_vector,w):
    ''''' Whenever lambda optimal is strictly positive,
    b should be s a unique value for all the training samples
    '''''
    non_zero_lambda = np.nonzero(lambda_vector)[0] # list of indexes of nonzero elements of lambda
    b = 0
    for elem in non_zero_lambda:
        b = b + ((1 - y[elem] * np.matmul(w.T, x[elem])) / y[elem])
    b = b / len(non_zero_lambda)
    return b